In [1]:
import matplotlib.pyplot as plt
import numpy
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model

In [2]:
def create_content_model(model = None, layer_name = None, layer_index = None):

    content_model = Model(inputs = model.input, outputs = model.get_layer(layer_name).output)
    
    return content_model

In [3]:
def create_style_model(model = None, layer_names = None):
    outputs = [model.get_layer(layer_name).output for layer_name in layer_names]
    style_model = Model(inputs = model.input, outputs = outputs)
    return style_model

In [4]:
def content_cost(content_model, content_img, generated_img):
    # Add a preprocess imgs for content and generated here
    #...

    # get the content image & generated img activations
    a_c = content_model(content_img)
    a_g = content_model(generated_img)

    cost = tf.reduce_mean(tf.square(a_c - a_g))

    return cost



In [5]:
def calculate_gram(activation = None):

    # Get activation dimensions
    _,height, width, channels = activation.shape

    # Unroll the activation/layer output
    a_unrolled = tf.reshape(activation, shape = [height * width, channels])
    gram = tf.matmul(a_unrolled, tf.transpose(a_unrolled))
    print(gram.shape)

In [6]:
def layer_style_cost(gram_s, gram_g):

    cost = tf.reduce_mean(tf.square(gram_s - gram_g))
    cost = cost / 4
    return cost

In [7]:
def style_cost(style_activations = None, generated_activations = None, layer_weights = None):

    current_cost = 0
    if layer_weights == None:
        layer_weights = [1. for layer in style_activations]

    for style_activation, generated_activation, weight in zip(style_activations, generated_activations, layer_weights):
        gram_g = calculate_gram(generated_activation)
        gram_s = calculate_gram(style_activation)
        current_cost += weight * layer_style_cost(gram_s, gram_g)
    
    cost = current_cost / len(style_activations)
    return cost





In [8]:
def train_loop(content_img, style_img, style_model = None, content_model = None, epochs = 30 ):
    pass

In [9]:
model = VGG19(input_shape = (128, 128, 3), include_top= False, weights = 'imagenet')
model.trainable = False
model.summary()

80134624/80134624 [==============================] - 0s 0us/step
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                              

In [10]:
m = create_content_model(model = model, layer_name = 'block2_conv2', layer_index = None)
_,height, width, channels = m.output_shape
calculate_gram(m.output)
print(type(m.output))

(4096, 4096)
<class 'keras.engine.keras_tensor.KerasTensor'>


In [13]:
layer_names = ['block2_conv2', 'block3_conv2', 'block4_conv2']
sm = create_style_model(model = model, layer_names = layer_names)
print(sm.output_shape)
print(type(sm.output))

[(None, 64, 64, 128), (None, 32, 32, 256), (None, 16, 16, 512)]
<class 'list'>
